# Python 과 DataBase 연동

1. 해당 라이브러리: DB 접속 드라이버, cx_Oracle, python-oracledb  
  - 설치:     
    1) Anaconda prompt 창(관리자 권한 실행) pip install 설치할 라이브러리  
    2) 주피터 노트북 안에서 설치 !pip install 설치할 라이브러리  
    3) Anaconda에서 pip install ~ or conda install ~  
  - pip install oracledb -- upgrade 최근 버전(권장)  
  - pip install cx_Oracle -- upgrade  
  
2. 접속정보 확인  
  - 사용자 계정과 비밀번호: scott/tiger  
  - 오라클 url: localhost:1521/xe  

In [ ]:
# 주피터 노트북에 라이브러리 설치
# !pip install cx_Oracle --upgrade

# DB 연동 작업

In [ ]:
# 라이브러리(모듈) 연결
import cx_Oracle as cx

## connection: 연결 통로 구축

In [ ]:
# db 연동 connect("사용자계정", "비밀번호", "url")
# url - localhost(127.0.0.1):1521/xe
conn = cx.connect("scott", "1234", "127.0.0.1:1521/xe")
# conn = cx.connect("scott/1234@localhost:1521/xe")

## DB에 테이블 조회

In [ ]:
# cursor(커서): 레코드 셋을 탐색하는 포인터
## 커서 객체 생성
cur = conn.cursor()

## query(질의문)
sql = "select * from product"

## 커서 객체를 이용하여 sql 실행
cur.execute(sql)

for row in cur:
    print(row)
    # clob 자료형은 그냥 읽어올 수 없다.
    description = row[3].read()
    print(description)

## 전체 레코드 삭제 후 테이블에 레코드 삽입

In [ ]:
sql = "delete from product"
cur.execute(sql)
conn.commit()

In [ ]:
# 파이썬에서 DB에 레코드 삽입
items = [
    (1, '레몬', 1500, '맛있는 레몬이 왔어요', 'lemon.jpg'),
    (2, '오렌지', 2500, '상큼한 오렌지 있어요', 'orange.jpg'),
    (3, '키위', 3500, '먹어도 키가 안 크는 키위가 있어요', 'kiwi.jpg'),
    (4, '포도', 2500, '포도가 맛있다.', 'grape.jpg'),
    (5, '딸기', 1800, '딸기도 좋아요', 'strawberry.jpg'),
    (6, '귤', 500, '제주도 대신 귤', 'tangerine.jpg')    
]

for row in items:
    # : 필드 인덱스(1부터)
    sql = "insert into product values (:1, :2, :3, :4, :5)"
    print(row)
    cur.execute(sql, row)
    
conn.commit()

In [ ]:
# 상품 갯수를 추출
sql = "select count(*) from product"
cur.execute(sql)

# 레코드 갯수가 1개일 경우
count = cur.fetchone()
print("상품갯수:", count[0])

In [ ]:
# 전체 레코드 삭제
sql = "delete from product"
cur.execute(sql)
conn.commit()

In [ ]:
# 레코드를 일괄적으로 추가하는 작업
items = [
    (1, '레몬', 1500, '맛있는 레몬이 왔어요', 'lemon.jpg'),
    (2, '오렌지', 2500, '상큼한 오렌지 있어요', 'orange.jpg'),
    (3, '키위', 3500, '먹어도 키가 안 크는 키위가 있어요', 'kiwi.jpg'),
    (4, '포도', 2500, '포도가 맛있다.', 'grape.jpg'),
    (5, '딸기', 1800, '딸기도 좋아요', 'strawberry.jpg'),
    (6, '귤', 500, '제주도 대신 귤', 'tangerine.jpg')    
]

sql = "insert into product values(:1, :2, :3, :4, :5)"
# 일괄적으로 레코드를 추가할 때는 레코드의 갯수를 지정해줘야 한다.
cur.bindarraysize = len(items)
cur.executemany(sql, items)
conn.commit()

In [ ]:
cur.close()
conn.close()

# 회원관리 프로그램

- CRUD(CREATE, SELECT, UPDATE, INSERT, DELETE) 즉, DML(조작어)
- 테이블 생성 Member(userid, userpwd, name), userid는 primary key 설정
- 주요기능: 회원가입(등록), 회원수정, 회원정보 출력, 회원삭제 등

In [1]:
import cx_Oracle as cx

conn = cx.connect("scott", "1234", "localhost:1521/xe")

## member 테이블 조회

In [7]:
cursor = conn.cursor()
cursor.execute("select * from member")

# 모든 레코드를 Resultset 객체에 저장
rs = cursor.fetchall() # list
# print(type(rs))
# print(rs)
for member in rs:
    print(member)

conn.commit()
conn.close() 


('admin', 'admin', '관리자')
('aaa', '1234', '이소라')
('ddd', '6485', 'djklcme')


In [ ]:
rs[0]

In [ ]:
userid = rs[0][0]
print(userid)

## 레코드 삽입

In [4]:
def insert(userid, userpass, username):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "insert into member values(:1, :2, :3)"
    
    cur = conn.cursor()
    cur.execute(sql, [userid, userpass, username])
    cur.close()
    conn.commit()
    conn.close()

In [5]:
print("회원가입할 회원의 정보를 입력하세요.")
userid = input('id =')
userpass = input('pass =')
username = input('name = ')

insert(userid, userpass, username)

회원가입할 회원의 정보를 입력하세요.
id =ddd
pass =6485
name = djklcme


## 레코드 수정

In [ ]:
def update(userid, userpass, username):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "update member set userpass=:1, username=:2 where userid =:3"
    
    cur = conn.cursor()
    cur.execute(sql, [userpass, username, userid])
    
    cur.close()
    conn.commit()
    conn.close()

In [ ]:
# id_check(userid)
def id_check(userid):
    conn = cx.connect("scott", "1234", "localhost:1521/xe")
    sql = "select * from member"
    
    cur = conn.cursor()
    cur.execute(sql)
    rs = cur.fetchall()
    
    for row in rs:
        if row[0] == userid:
            return True
        
        return False

In [ ]:
# 키보드를 통해 회원정보를 변경하는 작업
userid = input('id = ')
if id_check(userid):
    userpass = input('pass = ')
    username = input('name = ')
    update(userid, userpass, username)
else:
    print(f"{userid} 회원정보가 존재하지 않습니다.")

# 최종프로그램

## DB 연동

In [1]:
import cx_Oracle as cx

conn = cx.connect("scott", "1234", "localhost:1521/xe")
cur = conn.cursor()

## 기능별 함수 구현

## main processor

In [2]:
# 모듈 포함
from dbTest import *  # 함수 사용 시 모듈명 포함 필요 x

while True:
    print("\n== 회원관리 프로그램 ==")
    print("1.회원등록")
    print("2.회원수정")
    print("3.회원삭제")
    print("4.회원출력")
    print("5.작업종료")
    menu = int(input('메뉴 = '))
    
    if menu == 1:
        userid = input('id:')
        if id_check(userid) == 1:
            print(f"이미 존재하는 아이디입니다.")
        else:
            userpass = input('pass:')
            username = input('name:')
            insert(userid, userpass, username)
    elif menu == 2:
        userid = input('id:')
        if id_check(userid) == 0:
            print(f"{userid}의 회원정보가 존재하지 않습니다.")
        else:
            print("수정할 정보를 입력하세요.")
            userpass = input('pass:')
            username = input('name:')
            update(userid, userpass, username)
            print("수정이 완료되었습니다.")
    elif menu == 3:
        userid = input('id:')
        if id_check(userid) == 0:
            print(f"{userid}의 회원정보가 존재하지 않습니다.")
        else:
            delete(userid)
            print(f"{userid}회원 정보가 삭제되었습니다")
    elif menu == 4:
        listMember()
    elif menu == 5:
        close()  # 종료 전 자원 닫기
        break
    else:
        print("선택이 올바르지 않습니다.")
        
print("프로그램을 종료합니다.")


== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 2
id:bbb
수정할 정보를 입력하세요.
pass:1121
name:장보고
회원 정보가 수정되었습니다.
수정이 완료되었습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 4
('admin', 'admin', '관리자')
('aaa', '1234', '이소라')

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 1
id:aaa
이미 존재하는 아이디입니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 1
id:bbb
pass:4651
name:홍길동
회원 등록이 완료되었습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 2
id:ccc
ccc의 회원정보가 존재하지 않습니다.

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 3
id:bbb
bbb회원 정보가 삭제되었습니다

== 회원관리 프로그램 ==
1.회원등록
2.회원수정
3.회원삭제
4.회원출력
5.작업종료
메뉴 = 5
프로그램을 종료합니다.
